In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import pymorphy2
import re
analyzer = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_json("reviews1.json", encoding='utf8')
df.head()

,title,description
0,Анимационный ремейк «Гамлета» Уильяма Шекспира...,Главный мультфильм всех времен и народов (но л...
1,А на душе глубокий Шрам,В 1994 году ворвался в мировой кинематограф му...
2,Король лев — неповторимый шедевр на все времена,Начитавшись положительных отзывов о ремейке Ко...
3,Мультфильм на все времена!,Я долго откладывал этот мультфильм в долгий ящ...
4,Лучший на свете анимационный фильм,Анимационный фильм 'Король Лев' 1994 года – на...


In [4]:
def text_to_wordlist(text, remove_stopwords=False):
    # оставляем только буквенные символы, удаляем нечто в скобках
    text = re.sub("[^а-яА-ЯёЁ]"," ", re.sub(r'\((.*?)\)', "", text))
    # токенизируем текст и приводим к нижнему регистру
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    
    # удалим лишние части речи
    POS_black_list = ["NUMR", "NPRO", "PRED", "PREP", "CONJ", "PRCL", "INTJ"]
    filter_words = [word for word in words if analyzer.parse(word)[0].tag.POS not in POS_black_list]
    
    norm_words = [analyzer.parse(word)[0].normal_form for word in words]
    if remove_stopwords:
        # убираем стоп-слова
        stops = stopwords.words("russian") + ["это", "который", "наш", "мочь", "год", 
                                            "такой", "знать", "мы", "свой", "один", "другой", "хотеть",
                                            "всё", "весь", "очень", "думать", "нужно",
                                            "большой", "использовать", "говорить", "сказать",
                                            "иметь", "сделать", "первый", "каждый", "день", "её", "ваш",
                                            "стать", "больший", "ваше", "день", "самый", "понять",
                                            "просто", "ещё", "также", "например"]
        norm_words = [w for w in norm_words if w not in stops]
    return norm_words

In [5]:
df["preprocessed_text"] = df.apply(lambda row: text_to_wordlist(row["description"], remove_stopwords=True), axis=1)

In [6]:
df.head()

,title,description,preprocessed_text
0,Анимационный ремейк «Гамлета» Уильяма Шекспира...,Главный мультфильм всех времен и народов (но л...,"[главный, мультфильм, время, народ, производит..."
1,А на душе глубокий Шрам,В 1994 году ворвался в мировой кинематограф му...,"[ворваться, мировой, кинематограф, мультиплика..."
2,Король лев — неповторимый шедевр на все времена,Начитавшись положительных отзывов о ремейке Ко...,"[начитаться, положительный, отзыв, ремейка, ко..."
3,Мультфильм на все времена!,Я долго откладывал этот мультфильм в долгий ящ...,"[долго, откладывать, мультфильм, долгий, ящик,..."
4,Лучший на свете анимационный фильм,Анимационный фильм 'Король Лев' 1994 года – на...,"[анимационный, фильм, король, левый, взгляд, х..."


In [7]:
all_words = []
for doc in df["preprocessed_text"].tolist():
    all_words.extend(doc)

len(all_words)

32811

In [8]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(df["preprocessed_text"].tolist())
texts = df["preprocessed_text"].tolist()
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
from gensim.models import LdaMulticore
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=1)

In [10]:
lda_model.print_topics()

[(0,
  '0.021*"мультфильм" + 0.015*"король" + 0.009*"левый" + 0.008*"симб" + 0.006*"фильм" + 0.005*"ребёнок" + 0.005*"детство" + 0.005*"смотреть" + 0.005*"хороший" + 0.005*"персонаж"')]

In [11]:
!pip install pyLDAvis

You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Визуализируем темы

In [18]:
import pyLDAvis
from pyLDAvis import gensim_models
import pickle 
import os

ImportError: cannot import name 'gensim_models' from 'pyLDAvis' (c:\users\elya\appdata\local\programs\python\python37-32\lib\site-packages\pyLDAvis\__init__.py)

Тут была визуализация,но перестала работать :(

In [16]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

AttributeError: module 'pyLDAvis' has no attribute 'gensim_models'

В ходе работы было проведено тематическое моделирование (LDA) корпуса текстов. Перед
кластеризацией проведена предобработка документов: токенизация, лемматизация, нормализация, удаление стоп-слов, удаление служебных частей речи

Оптимальное количество тем равно 1, в противном случае, темы очень похожи, т.к. корпус дейтсвительно представляет собой отзывы о мультфильме "Король Лев".